In [11]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd
import tensorflow as tf

# Load the dataset
data_path = 'final_hcc_data_cleaned.csv'
data = pd.read_csv(data_path)

# Assume the last column is not needed for the model input
data = data.iloc[:, :-1]  # Exclude the last column

# Assume the last row of data is what we'll use for initial values
initial_values = data.iloc[-1].values

# Load the previously saved model
model = tf.keras.models.load_model('hcc_treatment_dqn_model.keras', compile=False)

# Try to compile the model with appropriate configurations
try:
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print("Model loaded and compiled with the optimizer state.")
except Exception as e:
    print(f"Error during model compilation: {str(e)}")

# Define labels from the dataset columns
labels = data.columns.tolist()

# Create input widgets for all features with specific labels and initial values
input_widgets = [widgets.FloatText(value=round(val, 2), description=str(label)) for val, label in zip(initial_values, labels)]

# Arrange widgets into three columns
column1 = widgets.VBox(input_widgets[:17])
column2 = widgets.VBox(input_widgets[17:34])
column3 = widgets.VBox(input_widgets[34:])

output_label = widgets.Label()  # To display predictions

def predict_treatment(button):
    # Collect inputs from all widgets and form an array of shape (1, number of features)
    input_values = np.array([[widget.value for widget in input_widgets]])
    
    try:
        prediction = model.predict(input_values)
        predicted_action = np.argmax(prediction[0])
        treatment_type = 'Aggressive' if predicted_action == 1 else 'Conservative'
        specific_treatment = "Surgery, Ablation, or Chemotherapy" if treatment_type == 'Aggressive' else "Sorafenib medication or Supportive care"
        output_label.value = f"Recommended Therapy: {treatment_type}, Specific Treatment: {specific_treatment}"
    except Exception as e:
        output_label.value = f"Error: {str(e)}"

# Button to trigger prediction
predict_button = widgets.Button(description='Predict Therapy')
predict_button.on_click(predict_treatment)

# Display widgets and output
widget_layout = widgets.HBox([column1, column2, column3])
display(widget_layout)
display(predict_button)
display(output_label)


Model loaded and compiled with the optimizer state.


Button(description='Predict Therapy', style=ButtonStyle())

Label(value='')

In [ ]:
!voila app2.ipynb